In [56]:
import pandas as pd
import numpy as np
import math

In [57]:
df = pd.read_csv("sine_wave.csv")
#print(df)
point_cloud_data = df.to_numpy()

In [58]:
dau = 0.1  # Minimum accuracy of point cloud
N = point_cloud_data.shape[0] # Total number of points
m = 5 # Empirical Value need to be checked for our case

In [59]:
x_min = min(point_cloud_data[:,0]) - dau
x_max = max(point_cloud_data[:,0]) + dau
y_min = min(point_cloud_data[:,1]) - dau
y_max = max(point_cloud_data[:,1]) + dau

In [60]:
S = ((x_max - x_min)*(y_max - y_min))/N
e = (m*S)**0.5 # cell length

In [61]:
n_x = math.ceil((x_max - x_min)/e)  # number of cells in x
n_y = math.ceil((y_max - y_min)/e)  # number of cells in y

In [62]:
def get_grid_id(i):
    point = point_cloud_data[i]
    return n_x * math.floor((point[1]-y_min)/e) + math.ceil((point[0] - x_min)/e)

In [63]:
num_grids = n_x * n_y;

In [74]:
grid_id2points = [[] for i in range(num_grids+1)]
for i in range (N):grid_id2points[get_grid_id(i)].append(i)

In [48]:
def is_valid(l):
    a,b = l
    return (a>=0 and a<=n_x-1 and b>=0 and b<=n_y-1)

def generation_points(i, n=1):
    a,b = (i-1)//32 , (i-1)%32
    l = [(a+of1,b+of2) for of2 in range(-n,n+1) for of1 in range(-n,n+1)]
    l = list(filter(is_valid,l))
    l = [a*32+b+1 for a,b in l]
    return l

In [49]:
def compute_r2(centre_point, point_id):
    pts = point_cloud_data[point_id]
    r_2 = (pts[0] - centre_point[0])**2 + (pts[1] - centre_point[1])**2 + (pts[2] - centre_point[2])**2
    return r_2

In [50]:
def snn(ctr_pt_id):
    snn_pt_ids, gen = []
    grid_id = get_grid_id(point_cloud_data[ctr_pt_id])
    centre_point = point_cloud_data[point_id]
    
    gen_id = 1;
    while(gen_id<3):
        gen = generation_points(grid_id, gen_id)
        r_2 = (gen_id * e)**2
        for grid_id in gen:
            point_ids = grid_id2points[grid_id]
            x = point_cloud_data[point_ids]-point_cloud_data[i]
            bool_array = np.sum(np.square(x),1) <= r_2
            snn_pt_ids.append(point_ids[bool_array])
            
        if(len(snn_pt_ids)):
            break
            
    return snn_pt_ids